In [3]:
class Instruction:
    def __init__(self, pc, opcode, operands=[]):
        self.pc = pc
        self.opcode = opcode
        self.operands = operands
        self.ready = True

class ROBEntry:
    def __init__(self, instruction):
        self.instruction = instruction
        self.ready = False

class ReorderBuffer:
    def __init__(self):
        self.entries = []

    def add_entry(self, instruction):
        entry = ROBEntry(instruction)
        self.entries.append(entry)
        return entry

    def commit(self):
        while self.entries and self.entries[0].ready:
            committed_instr = self.entries.pop(0)
            print(f"Committing instruction at PC {committed_instr.instruction.pc}")

class IssueQueue:
    def __init__(self):
        self.queue = []

    def add_instruction(self, instruction):
        self.queue.append(instruction)

    def issue(self):
        for instruction in self.queue:
            if instruction.ready:
                print(f"Issuing instruction at PC {instruction.pc}")
                self.queue.remove(instruction)
                return instruction
        return None

class ExecutionUnit:
    def __init__(self, registers, memory):
        self.current_instruction = None
        self.registers = registers
        self.memory = memory
        self.halted = False

    def execute(self, instruction):
        opcode = instruction.opcode
        operands = instruction.operands
        if opcode == "ADD":
            self.registers[operands[0]] = self.registers[operands[1]] + self.registers[operands[2]]
        elif opcode == "SUB":
            self.registers[operands[0]] = self.registers[operands[1]] - self.registers[operands[2]]
        elif opcode == "MUL":
            self.registers[operands[0]] = self.registers[operands[1]] * self.registers[operands[2]]
        elif opcode == "LOAD":
            self.registers[operands[0]] = self.memory[operands[1]]
        elif opcode == "STORE":
            self.memory[operands[1]] = self.registers[operands[0]]
        elif opcode == "EBREAK":
            print(f"EBREAK encountered at PC {instruction.pc}. Halting execution.")
            self.halted = True
        else:
            print(f"Unknown opcode: {opcode}")

        if not self.halted:
            print(f"Executed {opcode} instruction at PC {instruction.pc}")
        instruction.ready = True

class Processor:
    def __init__(self, ibuff):
        self.ibuff = ibuff
        self.rob = ReorderBuffer()
        self.issue_queue = IssueQueue()
        self.execution_unit = ExecutionUnit(registers={}, memory={})
        self.pc = 0

    def fetch(self):
        if self.pc < len(self.ibuff):
            instruction = self.ibuff[self.pc]
            print(f"Fetched instruction at PC {instruction.pc}")
            self.pc += 1
            return instruction
        return None

    def decode(self, instruction):
        print(f"Decoded instruction at PC {instruction.pc}")
        self.issue_queue.add_instruction(instruction)

    def run(self):
        while self.pc < len(self.ibuff) or self.issue_queue.queue:
            if self.execution_unit.halted:
                break

            # Fetch stage
            fetched_instr = self.fetch()
            if fetched_instr:
                # Decode stage
                self.decode(fetched_instr)
                # Add to ROB
                rob_entry = self.rob.add_entry(fetched_instr)
                fetched_instr.rob_entry = rob_entry

            # Issue stage
            issued_instr = self.issue_queue.issue()
            if issued_instr:
                # Execute stage
                self.execution_unit.execute(issued_instr)

            # Commit stage
            self.rob.commit()

# 示例指令集
ibuff = [
    Instruction(0, "ADD", ["r1", "r2", "r3"]),
    Instruction(4, "LOAD", ["r4", "0x1000"]),
    Instruction(8, "MUL", ["r5", "r6", "r7"]),
    Instruction(12, "SUB", ["r8", "r9", "r10"]),
    Instruction(16, "STORE", ["r1", "0x1004"]),
    Instruction(20, "EBREAK"),  
]

# 初始化寄存器和内存
initial_registers = {
    "r1": 0,
    "r2": 10,
    "r3": 20,
    "r4": 0,
    "r5": 5,
    "r6": 2,
    "r7": 3,
    "r8": 0,
    "r9": 50,
    "r10": 30,
}

initial_memory = {
    "0x1000": 42,
    "0x1004": 0,
}

# 初始化处理器
processor = Processor(ibuff)
processor.execution_unit.registers = initial_registers
processor.execution_unit.memory = initial_memory

# 运行处理器
processor.run()

# 打印最终寄存器和内存状态
print("Final Register State:", processor.execution_unit.registers)
print("Final Memory State:", processor.execution_unit.memory)


Fetched instruction at PC 0
Decoded instruction at PC 0
Issuing instruction at PC 0
Executed ADD instruction at PC 0
Fetched instruction at PC 4
Decoded instruction at PC 4
Issuing instruction at PC 4
Executed LOAD instruction at PC 4
Fetched instruction at PC 8
Decoded instruction at PC 8
Issuing instruction at PC 8
Executed MUL instruction at PC 8
Fetched instruction at PC 12
Decoded instruction at PC 12
Issuing instruction at PC 12
Executed SUB instruction at PC 12
Fetched instruction at PC 16
Decoded instruction at PC 16
Issuing instruction at PC 16
Executed STORE instruction at PC 16
Fetched instruction at PC 20
Decoded instruction at PC 20
Issuing instruction at PC 20
EBREAK encountered at PC 20. Halting execution.
Final Register State: {'r1': 30, 'r2': 10, 'r3': 20, 'r4': 42, 'r5': 6, 'r6': 2, 'r7': 3, 'r8': 20, 'r9': 50, 'r10': 30}
Final Memory State: {'0x1000': 42, '0x1004': 30}
